In [1]:
# import library
import string
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from util import JSONParser

In [2]:
# load data
path = "data/intents.json"

# buat objek JSONParser dan parse data intents.json
jp = JSONParser()
jp.parse(path)

# simpan dataframe dalam variabel df
df = jp.get_dataframe()

[INFO] Data JSON diubah ke DataFrame dengan bentuk : (315, 2)


In [3]:
# lihat 5 data pertama
df.head()

,text_input,intents
0,saya minta tolong,salam_pertanyaan
1,saya ada pertanyaan,salam_pertanyaan
2,Saya ingin bertanya,salam_pertanyaan
3,Mau tanya,salam_pertanyaan
4,Bantu saya,salam_pertanyaan


In [4]:
# hitung jumlah data per tag / inten
df.intents.value_counts()

intents
salam_umum                           16
bye_umum                             10
beasiswa_insan_mandiri                7
pembayaran_bank_ocbc                  7
beasiswa_cemerlang                    7
                                     ..
batas_waktu_pendaftaran_transfer      3
hubungi_humas_transfer                3
ujian_paket_c_lulusan_luar_negeri     3
pendaftaran_online_transfer           3
prosedur_sewa_stall_kantin            2
Name: count, Length: 64, dtype: int64

### Data preprocessing

- Mengubah semua alfabet menjadi huruf kecil
- Menghapus tanda baca

In [5]:
def preprocess(chat):
    # konversi ke lowercase
    chat = chat.lower()
    # menghapus tanda baca
    tandabaca = tuple(string.punctuation)
    chat = ''.join(ch for ch in chat if ch not in tandabaca)
    return chat

In [6]:
# implementasikan fungsi preprocess ke string
df['text_input_prep'] = df.text_input.apply(preprocess)

Apabila kita lihat hasilnya maka kita dapati hal berikut : 

In [7]:
df[['text_input', 'text_input_prep']].head(10)

,text_input,text_input_prep
0,saya minta tolong,saya minta tolong
1,saya ada pertanyaan,saya ada pertanyaan
2,Saya ingin bertanya,saya ingin bertanya
3,Mau tanya,mau tanya
4,Bantu saya,bantu saya
5,Hai,hai
6,Hi,hi
7,Halo,halo
8,Selamat Pagi,selamat pagi
9,Selamat Siang,selamat siang


### Vektorisasi

vektorisasi menggunakan metode bag of words yaitu mengumpulkan kata kata yang berasal dari korpus, dengan modul CountVectorizer dari scikit-learn

In [8]:
# inisiasi objek CountVectorizer
vect = CountVectorizer()

Langkah penting dalam metode *bag of words* adalah mengumpulkan *vocab* yang terdapat pada *corpus* yang kita miliki. Dalam scikit-learn kita dapat lakukan dengan cara :

In [9]:
# mengumpulkan vocab dari data teks yang sudah dilakukan praproses
vect.fit(df['text_input_prep'])

CountVectorizer()

In [10]:
# lihat list vocab
vect.get_feature_names_out()[:10] #batasi hanya 10 vocab teratas

array(['acara', 'ada', 'adakah', 'aja', 'akhir', 'akuntansi', 'anda',
       'antar', 'apa', 'apakah'], dtype=object)

Selanjutnya konversi data teks menjadi matriks sesuai vocab yang sudah dibuat

In [11]:
# ubah data teks menjadi matriks
text_vect = vect.transform(df.text_input_prep)

text_vect

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2202 stored elements and shape (315, 258)>

In [12]:
pd.DataFrame(text_vect.toarray(), columns=vect.get_feature_names_out())

,acara,ada,adakah,aja,akhir,akuntansi,anda,antar,apa,apakah,...,ulangnya,untuk,usia,va,waktu,wawancara,weh,woe,yang,you
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
311,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
312,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
313,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


### Modelling

Melakukan modelling dan data training menggunakan algoritma Multinomial Naive Bayes

In [13]:
# deklarasi objek MultinomialNB
nb = MultinomialNB()

# training data, dengan X : text_vect dan y : intents
nb.fit(text_vect, df.intents)

MultinomialNB()

model sudah dilatih, testing keluarkan hasil prediksi dari suatu input

In [14]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict(chat)

# tampilkan hasil prediksi
print(f"Hasil prediksi : {res[0]}")

Hasil prediksi : deadline_pendaftaran_s1_sarjana


prediksi dengan probabilitas jika hasil prediksi kurang dari threshold probabilitas = chatbot tidak mengerti

In [15]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict_proba(chat)

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

Max Prob : 0.10867033492033279
Max Index: 2
Label: beasiswa_cemerlang


### Efisiensi dengan Pipeline

Dari proses diatas kita bisa lihat apabila ada data teks maka kita perlu proses dalam dua langkah, yaitu vektorisasi dan pemodelan. Supaya proses menjadi lebih ringkas dan lebih mudah dalam proses deployment, kita akan buat pipeline

In [16]:
pipe = make_pipeline(CountVectorizer(),
                     MultinomialNB())

# Training
pipe.fit(df.text_input, df.intents)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

Dapat kita lihat dari proses training diatas, seolah-olah kita langsung memasukkan data teks dan labelnya langsung kedalam "*black box*" sehingga proses prediksi akan lebih ringkas.

Untuk inference dengan pipeline dapat kita lakukan dengan cara berikut : 

In [17]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# prediksi teks kedalam pipeline
res = pipe.predict_proba([chat])

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

Max Prob : 0.10095819560621322
Max Index: 52
Label: salam_umum


Dapat kita lihat bahwa teks baru setelah praproses bisa langsung masuk kedalam pipeline

### Simulasi Inference

Selanjutnya kita akan simulasikan chatbot mulai dari mendapatkan input sampai ke respon

Dalam kasus ini apabila intent yang terdeteksi adalah `bye` maka program berhenti

In [18]:
print("Anda Terhubung dengan chatbot Kami")
while True:
    # input user
    chat = input("Anda : ")
    # praproses input
    chat = preprocess(chat)
    
    # prediksi intent dengan pipeline
    res = pipe.predict_proba([chat])
    
    # ambil probabilitas tertinggi dan indeksnya
    max_prob = max(res[0])
    max_idx = np.argmax(res[0])
    intent = pipe.classes_[max_idx]
    
    # kondisi jika probabilitas kurang dari threshold
    if max_prob < 0.05:
        print("Bot : Maaf, saya tidak mengerti, jika anda butuh bantuan harap menghubungi humas kami.")
    else:
        # gunakan tag intent atau subintent yang terdeteksi untuk mendapatkan respons
        response = jp.get_response(intent)
        print(f"Bot : {response}")
    
    if intent == 'bye' or intent == 'bye_umum':
        break


Anda Terhubung dengan chatbot Kami
Bot : 👋 *Halo, saya asisten virtual UIB yang siap membantu Anda!* 

Jika Anda membutuhkan informasi tentang pendaftaran, beasiswa, atau hal lainnya, saya ada di sini untuk membantu.

Berikut beberapa hal yang bisa saya bantu:

✨ *Pendaftaran Program S1:* Semua informasi terkait pendaftaran, persyaratan, dan biaya.
✨ *Beasiswa:* Informasi tentang beasiswa yang dapat Anda ajukan.
✨ *Jadwal Penting:* Jangan lewatkan deadline pendaftaran dan acara penting lainnya.

💡 *Silakan ketik pertanyaan Anda*, saya akan memberikan jawaban yang Anda butuhkan!
Bot : 🌟 *Terima kasih telah menghubungi kami! Sampai jumpa!* 

Kami berharap Anda puas dengan bantuan yang diberikan. Jika ada pertanyaan lainnya atau jika Anda membutuhkan dukungan lebih lanjut, kami selalu siap membantu. Anda dapat menghubungi kami melalui:

📱 *LINE Pusat Informasi UIB:* [Link LINE]
📞 *WhatsApp Pusat Informasi UIB:* [Nomor WhatsApp]

Jangan ragu untuk kembali jika ada yang perlu Anda tanyakan!

Setelah kita berhasil simulasikan bot nya dalam notebook ini, kita simpan modelnya agar dapat dideploy dengan mudah

In [19]:
with open("chatbot_pipeline.pkl", "wb") as model_file:
    pickle.dump(pipe, model_file)